# Introduction

In [58]:
#importing all the things
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# A bit of data preprocessing

In [59]:
labels = pd.read_csv("/home/nate/CS82B_Final/training_solutions_rev1.csv") #replace with appropriate filename
print(labels.columns)
labels = labels.loc[:,:"Class1.2"]
print(labels.columns)
labels.set_index("GalaxyID", inplace=True)

Index(['GalaxyID', 'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2',
       'Class3.1', 'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2',
       'Class5.3', 'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2',
       'Class7.3', 'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5',
       'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1',
       'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3',
       'Class11.4', 'Class11.5', 'Class11.6'],
      dtype='object')
Index(['GalaxyID', 'Class1.1', 'Class1.2'], dtype='object')


In [60]:
print(labels.head)

<bound method NDFrame.head of           Class1.1  Class1.2
GalaxyID                    
100008    0.383147  0.616853
100023    0.327001  0.663777
100053    0.765717  0.177352
100078    0.693377  0.238564
100090    0.933839  0.000000
...            ...       ...
999948    0.510379  0.489621
999950    0.901216  0.098784
999958    0.202841  0.777376
999964    0.091000  0.909000
999967    0.767000  0.140000

[61578 rows x 2 columns]>


In [61]:
image_filepath = "/home/nate/CS82B_Final/images_training_rev1/" #replace with appropriate filepath

In [62]:
class_names = ["smooth", "features/disk"]

In [63]:
def get_label(galaxy):
    for i,x in enumerate(labels.loc[galaxy]):
        if x > 0.5: #there can't be multiple values greater than 0.5 and we want a good confidence for the label so I just chose that as the cutoff as well 
            return class_names[i]
    return pd.NA

In [64]:
def get_image(galaxy):
    image = Image.open(image_filepath + str(galaxy) + ".jpg")
    image = np.array(image.resize((32, 32)))
    return image

# One step of Data Analysis

## test_image = Image.open(image_filepath + "100008.jpg") 
print(np.array(test_image).shape)
plt.imshow(test_image)
plt.title(get_label(100008))

# Data preprocessing + data analysis


In [66]:
X_train, X_test, y_train, y_test = train_test_split(labels, labels, train_size=8000, random_state=42, test_size = 1600) 
#y values are useless cause they dont match with x values
#these are recreated/fixed below

temp = []
rem = []
count = 0
for x in X_train.index:
    a = get_label(x)
    if type(a) == type(pd.NA): 
        print("caught na") 
        count+=1
        rem.append(x)
    else:
        temp.append(a)
X_train = X_train.drop(rem)
y_train = pd.DataFrame(temp)
print(count)
print(y_train[0].value_counts())

temp = []
rem = []
for x in X_test.index:
    a = get_label(x)
    if type(a) == type(pd.NA):
        print("caught na")
        rem.append(x)
    else:
        temp.append(a)
X_test = X_test.drop(rem)
y_test = pd.DataFrame(temp)

X_train_knn= pd.DataFrame([get_image(x).flatten() for x in X_train.index])
X_train= torch.tensor(np.array([get_image(x) for x in X_train.index]))


X_test_knn = pd.DataFrame([get_image(x).flatten() for x in X_test.index])
X_test = torch.tensor(np.array([get_image(x) for x in X_test.index]))

caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na
caught na


# ML (knn)

In [67]:
hyperparameters = {'n_neighbors': [3, 7, 11, 15],
                   'weights': ['uniform', 'distance'],
                   'p': [1, 2]}
knn = KNeighborsClassifier()
rs = RandomizedSearchCV(knn, hyperparameters, scoring='f1_micro', n_jobs=-1, n_iter = 7)
rs.fit(X_train_knn, np.ravel(y_train))
model = rs.best_estimator_
print(rs.best_params_)

{'weights': 'distance', 'p': 1, 'n_neighbors': 11}


In [68]:
pred = model.predict(X_test_knn)
print("Accuracy score ", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

Accuracy score  0.7464698331193839
               precision    recall  f1-score   support

features/disk       0.79      0.73      0.76       860
       smooth       0.70      0.76      0.73       698

     accuracy                           0.75      1558
    macro avg       0.75      0.75      0.75      1558
 weighted avg       0.75      0.75      0.75      1558



In [69]:
not_optimal = KNeighborsClassifier(n_neighbors = 3, weights = "distance", p = 2)
not_optimal.fit(X_train_knn, np.ravel(y_train))

pred = not_optimal.predict(X_test_knn)
print("Accuracy Score: ", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

Accuracy Score:  0.6976893453145058
               precision    recall  f1-score   support

features/disk       0.74      0.69      0.72       860
       smooth       0.65      0.71      0.68       698

     accuracy                           0.70      1558
    macro avg       0.70      0.70      0.70      1558
 weighted avg       0.70      0.70      0.70      1558



# CNN

In [92]:
#setting up cnn archetictureclass CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn = CNN()

In [93]:
#trainingcel = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)
cnn.train()

numeric_train = [class_names.index(y) for y in y_train[0]]
X_train = X_train.float()

for x in range(5):
    for x in range(0, len(X_train), 64):
        galaxies = X_train[x:x+64].permute(0, 3, 1, 2)
        labels = torch.tensor(numeric_train[x:x+64])

        optimizer.zero_grad()
        values = cnn(galaxies)
        loss = cel(values, labels)
        loss.backward()
        optimizer.step()

In [94]:
numeric_test = [class_names.index(y) for y in y_test[0]]
X_test = X_test.float()

In [102]:
#testingcnn.eval()
total=0
correct = 0
pred = []
with torch.no_grad():
    for x in range(0, len(X_test), 64):
        galaxies = X_test[x:x+64].permute(0, 3, 1, 2)
        labels = torch.tensor(numeric_test[x:x+64])

        outputs = cnn(galaxies)
        _, predicted = torch.max(outputs, 1) 
        for x in predicted:
            pred.append(class_names[x.item()])
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(classification_report(y_test, pred))
print(f"Accuracy score: " + str(accuracy))

               precision    recall  f1-score   support

features/disk       0.81      0.85      0.83       860
       smooth       0.80      0.75      0.77       698

     accuracy                           0.80      1558
    macro avg       0.80      0.80      0.80      1558
 weighted avg       0.80      0.80      0.80      1558

Accuracy score: 0.8048780487804879
